# Creating and Valuing a CDS Contract vs Markit

Example CDS Valuation and comparison with market standard ISDA model on Markit website

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [1]:
from financepy.finutils import *
from financepy.products.libor import *
from financepy.products.credit import *

###################################################################
# FINANCEPY BETA Version 0.180 - This build: 22 Sep 2020 at 22:13 #
#     This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy #
###################################################################



## Creating a CDS Contract

This based on an example from Markit's CDS calculator website https://www.markit.com/markit.jsp?jsppage=pv.jsp

In [2]:
tradeDate = FinDate(21, 11, 2019)
settlementDate = tradeDate.addDays(1)
maturityDate = FinDate(20, 12, 2024)
cdsCoupon = 0.050
notional = ONE_MILLION
longProtection = True
tradeDate = FinDate(2019, 8, 9)

In [3]:
cdsContract = FinCDS(settlementDate, maturityDate, cdsCoupon, notional, longProtection)

In [4]:
print(cdsContract)

OBJECT TYPE: FinCDS
STEP-IN DATE: FRI 22 NOV 2019
MATURITY: FRI 20 DEC 2024
NOTIONAL: 1000000
RUNNING COUPON: 500.0bp
DAYCOUNT: FinDayCountTypes.ACT_360
FREQUENCY: FinFrequencyTypes.QUARTERLY
CALENDAR: FinCalendarTypes.WEEKEND
BUSDAYRULE: FinBusDayAdjustTypes.FOLLOWING
DATEGENRULE: FinDateGenRuleTypes.BACKWARD
ACCRUED DAYS: 63
PAYMENT_DATE, YEAR_FRAC, FLOW
FRI 20 SEP 2019,     0.000000,     0.000000
FRI 20 DEC 2019,     0.252778, 12638.888889
FRI 20 MAR 2020,     0.252778, 12638.888889
MON 22 JUN 2020,     0.261111, 13055.555556
MON 21 SEP 2020,     0.252778, 12638.888889
MON 21 DEC 2020,     0.252778, 12638.888889
MON 22 MAR 2021,     0.252778, 12638.888889
MON 21 JUN 2021,     0.252778, 12638.888889
MON 20 SEP 2021,     0.252778, 12638.888889
MON 20 DEC 2021,     0.252778, 12638.888889
MON 21 MAR 2022,     0.252778, 12638.888889
MON 20 JUN 2022,     0.252778, 12638.888889
TUE 20 SEP 2022,     0.255556, 12777.777778
TUE 20 DEC 2022,     0.252778, 12638.888889
MON 20 MAR 2023,     0.25

## Build Libor Curve

In [5]:
dcType = FinDayCountTypes.ACT_360
depo1 = FinLiborDeposit(settlementDate, "1M", 0.017156, dcType)
depo2 = FinLiborDeposit(settlementDate, "2M", 0.018335, dcType)
depo3 = FinLiborDeposit(settlementDate, "3M", 0.018988, dcType)
depo4 = FinLiborDeposit(settlementDate, "6M", 0.018911, dcType)
depo5 = FinLiborDeposit(settlementDate, "12M", 0.019093, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

swapType = FinLiborSwapTypes.PAYER
dcType = FinDayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swap1 = FinLiborSwap(settlementDate,"2Y",swapType,0.015630,fixedFreq,dcType)
swap2 = FinLiborSwap(settlementDate,"3Y",swapType,0.015140,fixedFreq,dcType)
swap3 = FinLiborSwap(settlementDate,"4Y",swapType,0.015065,fixedFreq,dcType)
swap4 = FinLiborSwap(settlementDate,"5Y",swapType,0.015140,fixedFreq,dcType)
swap5 = FinLiborSwap(settlementDate,"6Y",swapType,0.015270,fixedFreq,dcType)
swap6 = FinLiborSwap(settlementDate,"7Y",swapType,0.015470,fixedFreq,dcType)
swap7 = FinLiborSwap(settlementDate,"8Y",swapType,0.015720,fixedFreq,dcType)
swap8 = FinLiborSwap(settlementDate,"9Y",swapType,0.016000,fixedFreq,dcType)
swap9 = FinLiborSwap(settlementDate,"10Y",swapType,0.016285,fixedFreq,dcType)
swap10 = FinLiborSwap(settlementDate,"12Y",swapType,0.01670,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9,swap10]

liborCurve = FinLiborCurve(settlementDate, depos, [], swaps)

# Build a CDS Curve

In [6]:
cdsFlatSpread = 0.0100

In [7]:
cds1 = FinCDS(settlementDate, "1Y", cdsFlatSpread)
cds2 = FinCDS(settlementDate, "2Y", cdsFlatSpread)
cds3 = FinCDS(settlementDate, "3Y", cdsFlatSpread)
cds4 = FinCDS(settlementDate, "4Y", cdsFlatSpread)
cds5 = FinCDS(settlementDate, "5Y", cdsFlatSpread)
cds6 = FinCDS(settlementDate, "7Y", cdsFlatSpread)
cds7 = FinCDS(settlementDate, "10Y", cdsFlatSpread)
cds8 = FinCDS(settlementDate, "15Y", cdsFlatSpread)

In [14]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [15]:
recoveryRate = 0.40

In [16]:
issuerCurve = FinCDSCurve(settlementDate, cdss, liborCurve, recoveryRate)

# Valuation Results

In [17]:
spd = cdsContract.parSpread(settlementDate, issuerCurve, recoveryRate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD  100.00000 bp


In [18]:
v = cdsContract.value(settlementDate, issuerCurve, recoveryRate)
full_pv = v['full_pv']
clean_pv = v['clean_pv']

print("FULL VALUE  %12.2f"% full_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

FULL VALUE    -198583.98
CLEAN VALUE   -189833.98


MARKIT CALCULATE -198,551 for the FULL VALUE

In [19]:
cleanp = cdsContract.cleanPrice(settlementDate, issuerCurve, recoveryRate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE   118.983359


MARKIT CALCULATE 118.98%

In [20]:
accruedDays = cdsContract.accruedDays()
print("ACCRUED_DAYS", accruedDays)

ACCRUED_DAYS 63


MARKIT CALCULATE 63

In [21]:
accruedInterest = cdsContract.accruedInterest()
print("ACCRUED_COUPON", accruedInterest)

ACCRUED_COUPON -8750.0


MARKIT CALCULATE 8750

In [22]:
protPV = cdsContract.protectionLegPV(settlementDate, issuerCurve, recoveryRate)
print("PROTECTION_PV", protPV)

PROTECTION_PV 47458.49444861145


In [23]:
premPV = cdsContract.premiumLegPV(settlementDate, issuerCurve, recoveryRate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 246042.4723781413


## Risk Measures

In [24]:
creditDV01 = cdsContract.creditDV01(settlementDate, issuerCurve, recoveryRate)
print("Credit DV01  %12.2f"% creditDV01)

Credit DV01        553.63


MARKIT FOUND 554

In [25]:
interestDV01 = cdsContract.interestDV01(settlementDate, issuerCurve, recoveryRate)
print("Interest DV01  %12.2f"% interestDV01)

Interest DV01         48.82


MARKIT FOUND 49

## Full Analysis 

In [26]:
cdsContract.printFlows(issuerCurve)

PAYMENT_DATE      YEAR_FRAC      FLOW           DF       SURV_PROB      NPV
FRI 20 DEC 2019   0.252778     12638.89     0.998668     0.998704     12605.69
FRI 20 MAR 2020   0.252778     12638.89     0.993776     0.994505     12491.20
MON 22 JUN 2020   0.261111     13055.56     0.988911     0.990186     12784.08
MON 21 SEP 2020   0.252778     12638.89     0.984174     0.986023     12265.00
MON 21 DEC 2020   0.252778     12638.89     0.980286     0.981877     12165.18
MON 22 MAR 2021   0.252778     12638.89     0.978268     0.977756     12089.19
MON 21 JUN 2021   0.252778     12638.89     0.975718     0.973652     12007.06
MON 20 SEP 2021   0.252778     12638.89     0.971969     0.969565     11910.73
MON 20 DEC 2021   0.252778     12638.89     0.968307     0.965496     11816.05
MON 21 MAR 2022   0.252778     12638.89     0.964822     0.961444     11724.11
MON 20 JUN 2022   0.252778     12638.89     0.961404     0.957409     11633.56
TUE 20 SEP 2022   0.255556     12777.78     0.958086   